# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
weather_csv = "../starter_code/city_data.csv"
weatherDF = pd.read_csv(weather_csv)
weatherDF.head()

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,butaritari,KI,81.55,75,12,17.87,3.0707,172.7902,1658860310
1,atasu,KZ,79.88,20,41,15.66,48.6814,71.6433,1658860310
2,nikolskoye,RU,74.80,70,83,6.91,59.7035,30.7861,1658860311
3,airai,TL,53.29,70,100,3.67,-8.9266,125.4092,1658860311
4,east london,ZA,59.94,82,0,8.05,-33.0153,27.9116,1658860120


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure api key
gmaps.configure(api_key=g_key)

weatherDF = weatherDF.dropna()
humidity = weatherDF["Humidity"].astype(float)

# Extract latitude and longitude
locations = weatherDF[["Latitude", "Longitude"]].astype(float)

# Create a Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weatherDF["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Filter by weather conditions (above 70 and below 80, windspeed below 10 mph, and 0% cloudiness)
filteredDF = weatherDF.loc[(weatherDF["Temperature"] < 80) & (weatherDF["Temperature"] > 70)]
filteredDF = filteredDF.loc[filteredDF["Windspeed"] < 10]
filteredDFf = filteredDF.loc[filteredDF["Cloudiness"] == 0]

# Drop any rows with null values
filteredDF = filteredDF.dropna()

filteredDF

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
2,nikolskoye,RU,74.80,70,83,6.91,59.7035,30.7861,1658860311
5,buala,SB,78.26,84,83,4.54,-8.1450,159.5921,1658860312
7,iquitos,PE,77.04,94,75,1.14,-3.7481,-73.2472,1658860312
23,guerrero negro,MX,79.18,52,69,6.35,27.9769,-114.0611,1658860101
25,jamestown,US,77.95,56,40,3.00,42.0970,-79.2353,1658860062
...,...,...,...,...,...,...,...,...,...
538,uige,AO,72.27,26,55,4.63,-7.6087,15.0613,1658860502
542,gulariya,NP,79.07,84,100,5.35,28.2333,81.3333,1658860503
548,moram,IN,73.94,87,100,4.90,17.8000,76.4667,1658860506
553,aksarayskiy,RU,78.01,45,97,3.29,46.7924,48.0119,1658860508


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = filteredDF

# Create parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Get response using request.get
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Skipping.")
    
    print("------------------------------------------")

Retrieving Results for Index 2: nikolskoye.
Closest hotel is Tourist House - Sablino.
------------------------------------------
Retrieving Results for Index 5: buala.
Closest hotel is Maringe Lagoon Lodge.
------------------------------------------
Retrieving Results for Index 7: iquitos.
Closest hotel is Cortez Malaga Techy Caroly.
------------------------------------------
Retrieving Results for Index 23: guerrero negro.
Closest hotel is Casa Laguna, Bed & Breakfast.
------------------------------------------
Retrieving Results for Index 25: jamestown.
Closest hotel is Hampton Inn & Suites Jamestown.
------------------------------------------
Retrieving Results for Index 33: carutapera.
Closest hotel is Lidera.
------------------------------------------
Retrieving Results for Index 40: sao filipe.
Closest hotel is Hotel Xaguate.
------------------------------------------
Retrieving Results for Index 43: brigantine.
Closest hotel is Legacy Vacation Resorts - Brigantine Beach.
-------

Closest hotel is Golden Berries Hotel and Convention.
------------------------------------------
Retrieving Results for Index 385: gamba.
Skipping.
------------------------------------------
Retrieving Results for Index 390: pindiga.
Closest hotel is Accommodation Center.
------------------------------------------
Retrieving Results for Index 391: cerkezkoy.
Closest hotel is City Hotel.
------------------------------------------
Retrieving Results for Index 398: kropotkin.
Closest hotel is Hotel Europe Kropotkin.
------------------------------------------
Retrieving Results for Index 399: lompoc.
Closest hotel is Holiday Inn Express Lompoc, an IHG Hotel.
------------------------------------------
Retrieving Results for Index 405: tumaco.
Closest hotel is Unión Victoria.
------------------------------------------
Retrieving Results for Index 406: yenagoa.
Closest hotel is Aridolf Resort Wellness and Spa.
------------------------------------------
Retrieving Results for Index 407: birao.

In [18]:
hotel_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date,Hotel Name
2,nikolskoye,RU,74.80,70,83,6.91,59.7035,30.7861,1658860311,Tourist House - Sablino
5,buala,SB,78.26,84,83,4.54,-8.1450,159.5921,1658860312,Maringe Lagoon Lodge
7,iquitos,PE,77.04,94,75,1.14,-3.7481,-73.2472,1658860312,Cortez Malaga Techy Caroly
23,guerrero negro,MX,79.18,52,69,6.35,27.9769,-114.0611,1658860101,"Casa Laguna, Bed & Breakfast"
25,jamestown,US,77.95,56,40,3.00,42.0970,-79.2353,1658860062,Hampton Inn & Suites Jamestown
...,...,...,...,...,...,...,...,...,...,...
538,uige,AO,72.27,26,55,4.63,-7.6087,15.0613,1658860502,Hotel Catraio
542,gulariya,NP,79.07,84,100,5.35,28.2333,81.3333,1658860503,PUJA TRADING
548,moram,IN,73.94,87,100,4.90,17.8000,76.4667,1658860506,Bangi Farm House Murum
553,aksarayskiy,RU,78.01,45,97,3.29,46.7924,48.0119,1658860508,NaN


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [21]:
# Add marker layers ontop of heat map
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))